In [17]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_community.document_loaders.text import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pydantic import BaseModel ,Field
from langchain_core.output_parsers import JsonOutputParser
from langchain.prompts import PromptTemplate
from dotenv import dotenv_values
import json
import pandas as pd

# Load "gemini-pro"

In [2]:
env_values = dotenv_values(".env")
Gimni_API_KEY = env_values['Gimni_API_KEY']

In [3]:
llm = GoogleGenerativeAI(model="gemini-pro", google_api_key=Gimni_API_KEY,temperature=0.5)

# Load Extracted Text 

In [4]:
loader = TextLoader('Tesseract_output/output_text/Enviromental_factors_egypt_arabic/all_text.txt')
all_text=loader.load()

In [5]:
print(all_text[0].page_content)

مصرء أرض الأهرامات الشاهقة والعجائب القديمة» تتمتع ببيئة فريدة تشكلت من خلال التفاعل المعقد بين العوامل الحية
(الحيوية) والعوامل غير الحية (اللاأحيائية). فهم هذه العوامل ضروري لتقدير التوازن البيئي الدقيق لمصر والتحديات التي
.تواجهها

أرض التناقضات: وفرة حيوية وجفاف

على الرغم من المناخ الصحراوي القاسيء تدعم مصر مجموعة متنوعة من الحياة النباتية والحيوانية. تتكيف أشجار الأكاسيا
بقدرتها على حفظ المياه مع الرمال الصحراويةء بينما تنمو نباتات البردي ونخيل التمر على طول نهر النيل الذي يمنح الحياة»
مما يوفر الغذاء والمأوى لمجموعة متنوعة من الحيوانات. هذه النظم البيئية النابضة بالحياة تظهر مرونة الحياة الرائعة في
.مصر

نهر النيل: شريان الحياة لمصر

نهر النيل هو قلب النظام البيئي لمصرء شريان حيوي حافظ على السكان والزراعة لآلاف السنين. كانت فيضاناته السنوية تودع
الطمي الخصيب. مما يخلق دلتا النيل» وهي منطقة زراعية حيوية. هذه الشريط الخصيب من الأرض وسط الصحراء الشاسعة
يبرز أهمية النيل للحياة في مصر. ومع ذلك. فإن بناء السدود غيّر الدورة الفيضانية الطبيعية» مما أثر على تجديد التربة
.الخصبة وتقليل تد

# Split Text into Chunks

In [6]:
text_splitter = RecursiveCharacterTextSplitter(
    # Set a really small chunk size, just to show.
    chunk_size=2000,
    chunk_overlap=200,
    length_function=len,
    # is_separator_regex=False,
    separators=[
        "\n\n",
        "\n",
        " ",
        ".",
        ",",
        "\u200b",  # Zero-width space
        "\uff0c",  # Fullwidth comma
        "\u3001",  # Ideographic comma
        "\uff0e",  # Fullwidth full stop
        "\u3002",  # Ideographic full stop
        "",
    ],
)

In [7]:
query_1="Enviromental_factors_egypt_arabic"
meta_data={"document":query_1}
chunks = text_splitter.create_documents([all_text[0].page_content],metadatas=[meta_data])


In [8]:
len(chunks)

3

In [9]:
print (chunks[1].page_content)

يؤدي تغير المناخ إلى اضطراب الأنماط الجوية» مما يؤثر على التوازن الدقيق لأنظمة مصر البيئية. يمكن أن تؤدي الزيادة في
درجات الحرارة العالمية إلى تفاقم التصحرء مما يقلل من إنتاجية الزراعة. بالإضافة إلى ذلكء» يمكن أن تؤدي التغيرات في
أنماط هطول الأمطار إلى اضطراب الموارد المائية المحدودة بالفعل في البلاد. يمكن أيضًا للأنشطة البشرية مثل الرعي الجائر
.والممارسات الزراعية غير المستدامة أن تهدد التوازن الدقيق لأنظمة الصحراء البيئية وتقلل من الأراضي الصالحة للزراعة
الكشف عن عظمة النيل


نهر النيل العظيم: الكشف عن عظمته بالأرقام

يمتد نهر النيل على طول 6,650 كيلومترًا (4,132 ميلا)» وهو أطول نهر في إفريقيا. بمتوسط تدفق يبلغ 2,830 متر مكعب
في الثانية (99,500 قدم مكعب في الثانية)» يعد مصدرً! رئيسيًا للمياه العذبة لمصر. دلتا النيل» التي تشكلت من ترسبات النهر
على مدى آلاف السنين؛ تمتد على حوالي 24,000 كيلومتر مربع (9,300 ميل مربع)؛ وهي منطقة خصبة حيوية للزراعة.
ومع ذلكء فإن بناء السدود على طول النيل غيّر الدورة الفيضانية الطبيعية» مما أتر على تجديد التربة الخصبة وقد يعرقل توافر
.المياه في المستقبل بس

# Create output parser for the prompt

In [10]:
# Define your desired data structure.
class pairs(BaseModel):
    question: str = Field(description="An Arabic question identified within a text.")
    answer: str = Field(description="An Arabic answer identified within a text. ")
    context: str = Field(description="An Arabic context that can be used to answer the question identified within a text.")

In [11]:

parser = JsonOutputParser(pydantic_object=pairs)

In [12]:
temp="\n".join([
   "You are an API that converts bodies of Arabic text into questions , answers and context in Arabic .",
   
   "{format_instructions}",
   
   
    "The question must be in question format." ,
    "Make the answers a little bit longer.",
    "The answer must exist in the context.",
    "The context must be  a little bit longer than The answer .",
    "At least 50 questions and answers.",
    "### text:",
    "{text}",
])

prompt=PromptTemplate(
    template=temp,
    input_variables=["text"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [23]:
result_all=[]
for chunk in chunks:
    message_all = prompt.format(text=chunk.page_content)
    result=llm( message_all )
    result_all.append(result.split("\n"))

In [24]:
result_all

[['{"question": "ما العوامل التي يجب فهمها لتقدير التوازن البيئي لمصر؟", "answer": "العوامل الحية (الحيوية) والعوامل غير الحية (اللاأحيائية) التي تتفاعل لتشكل بيئة مصر الفريدة.", "context": "فهم هذه العوامل ضروري لتقدير التوازن البيئي الدقيق لمصر والتحديات التي تواجهها."},',
  '{"question": "ما الذي يميز أشجار الأكاسيا في مصر؟", "answer": "قدرتها على حفظ المياه والتكيف مع الرمال الصحراوية القاسية.", "context": "تتكيف أشجار الأكاسيا بقدرتها على حفظ المياه مع الرمال الصحراويةء بينما تنمو نباتات البردي ونخيل التمر على طول نهر النيل الذي يمنح الحياة» مما يوفر الغذاء والمأوى لمجموعة متنوعة من الحيوانات."},',
  '{"question": "ما أهمية نهر النيل بالنسبة لمصر؟", "answer": "نهر النيل هو شريان الحياة لمصر، حيث حافظ على السكان والزراعة لآلاف السنين من خلال فيضاناته السنوية التي تودع الطمي الخصيب.", "context": "نهر النيل هو قلب النظام البيئي لمصرء شريان حيوي حافظ على السكان والزراعة لآلاف السنين. كانت فيضاناته السنوية تودع الطمي الخصيب. مما يخلق دلتا النيل» وهي منطقة زراعية حيوية."},',
  '{"questi

# Reformat the output to CSV file

In [25]:
flattened_data = []
for sublist in result_all:
    flattened_data.extend(sublist)


In [26]:
flattened_data

['{"question": "ما العوامل التي يجب فهمها لتقدير التوازن البيئي لمصر؟", "answer": "العوامل الحية (الحيوية) والعوامل غير الحية (اللاأحيائية) التي تتفاعل لتشكل بيئة مصر الفريدة.", "context": "فهم هذه العوامل ضروري لتقدير التوازن البيئي الدقيق لمصر والتحديات التي تواجهها."},',
 '{"question": "ما الذي يميز أشجار الأكاسيا في مصر؟", "answer": "قدرتها على حفظ المياه والتكيف مع الرمال الصحراوية القاسية.", "context": "تتكيف أشجار الأكاسيا بقدرتها على حفظ المياه مع الرمال الصحراويةء بينما تنمو نباتات البردي ونخيل التمر على طول نهر النيل الذي يمنح الحياة» مما يوفر الغذاء والمأوى لمجموعة متنوعة من الحيوانات."},',
 '{"question": "ما أهمية نهر النيل بالنسبة لمصر؟", "answer": "نهر النيل هو شريان الحياة لمصر، حيث حافظ على السكان والزراعة لآلاف السنين من خلال فيضاناته السنوية التي تودع الطمي الخصيب.", "context": "نهر النيل هو قلب النظام البيئي لمصرء شريان حيوي حافظ على السكان والزراعة لآلاف السنين. كانت فيضاناته السنوية تودع الطمي الخصيب. مما يخلق دلتا النيل» وهي منطقة زراعية حيوية."},',
 '{"question":

In [28]:

cleaned_data = []
for json_str in flattened_data:
    try:
        
        cleaned_str = json_str.strip().rstrip(',')
        cleaned_data.append(json.loads(cleaned_str))
    except json.JSONDecodeError as e:
        print(f"Error parsing JSON: {e}\nJSON String: {json_str}")


Error parsing JSON: Unterminated string starting at: line 1 column 84 (char 83)
JSON String: {"question": "ما هي بعض الطرق التي يمكن بها للناس حماية البيئة في مصر؟", "answer": "يمكن للناس حماية البي
Error parsing JSON: Unterminated string starting at: line 1 column 182 (char 181)
JSON String: {"question": "ما هي التحديات التي تواجه مصر بسبب ندرة الأمطار في الصحراء الكبرى؟", "answer": "يمكن أن تخلق ندرة الأمطار في الصحراء الكبرى بيئة صعبة لمعظم أشكال الحياة.", "context": "احتضان الصحراء الكبرى: منظر طبيعي يتسم بالظروف القاسية تغطي الصحراء الكبرى» التي تشمل جزءًا كبيرًا من مساحة مصرء بيئة قاسية مع ندرة في الأمطار ودرجات حرارة متطرفة. هذه الصحراء الشاسعة» التي تعد أكبر صحراء حارة في العالم» تغطي حوالي 9.4 مليون كيلومتر مربع (3.6 مليون ميل مربع). يبلغ متوسط هطول الأمطار السنوي في الصح


In [29]:
cleaned_data

[{'question': 'ما العوامل التي يجب فهمها لتقدير التوازن البيئي لمصر؟',
  'answer': 'العوامل الحية (الحيوية) والعوامل غير الحية (اللاأحيائية) التي تتفاعل لتشكل بيئة مصر الفريدة.',
  'context': 'فهم هذه العوامل ضروري لتقدير التوازن البيئي الدقيق لمصر والتحديات التي تواجهها.'},
 {'question': 'ما الذي يميز أشجار الأكاسيا في مصر؟',
  'answer': 'قدرتها على حفظ المياه والتكيف مع الرمال الصحراوية القاسية.',
  'context': 'تتكيف أشجار الأكاسيا بقدرتها على حفظ المياه مع الرمال الصحراويةء بينما تنمو نباتات البردي ونخيل التمر على طول نهر النيل الذي يمنح الحياة» مما يوفر الغذاء والمأوى لمجموعة متنوعة من الحيوانات.'},
 {'question': 'ما أهمية نهر النيل بالنسبة لمصر؟',
  'answer': 'نهر النيل هو شريان الحياة لمصر، حيث حافظ على السكان والزراعة لآلاف السنين من خلال فيضاناته السنوية التي تودع الطمي الخصيب.',
  'context': 'نهر النيل هو قلب النظام البيئي لمصرء شريان حيوي حافظ على السكان والزراعة لآلاف السنين. كانت فيضاناته السنوية تودع الطمي الخصيب. مما يخلق دلتا النيل» وهي منطقة زراعية حيوية.'},
 {'question

In [30]:

df = pd.DataFrame(cleaned_data)

In [31]:
df.to_csv("dataset_context.csv")

In [33]:
print(df["context"].iloc[5])

بالاعتراف بأهمية بيئتها الفريدة» قامت مصر بمبادرات متنوعة للحفظ. تم إنشاء مناطق محمية لحماية الحياة البرية والموائل الحيوية. بالإضافة إلى ذلك؛ تُبذل جهود لتعزيز الممارسات الزراعية المستدامة التي تحافظ على المياه وتقلل من التصحر.
